In [2]:
!pip install scattertext
!pip install spacy

In [3]:
import scattertext as st
import pandas as pd
import spacy
from scattertext import SampleCorpora, PhraseMachinePhrases, dense_rank, RankDifference, AssociationCompactor, produce_scattertext_explorer
from scattertext.CorpusFromPandas import CorpusFromPandas
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

2023-04-11 15:48:55.668349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
df = pd.read_csv('/Users/Link/Documents/5005数据可视化/Group project/archive/kindle_reviews.csv')
df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


**Columns:**  
asin - 书籍id  
helpful - 有帮助的概率    
overall - 书籍评分  
reviewText - 评论文本  
reviewTime - 评论时间   
reviewerID - 评论者id   
reviewerName - 评论者名称  
summary - 评论综述  
unixReviewTime - 时间戳

In [5]:
# 定义函数来根据分数评价积极或消极
def get_evaluation(score):
    if score >= 4:
        return 'pos'
    else:
        return 'neg'

# 将函数应用到每一行的分数列中，并将结果存储到评价列中
df['sentiment'] = df['overall'].apply(get_evaluation)

df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,sentiment
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000,pos
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400,pos
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600,pos
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000,pos
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200,pos


In [6]:
#test = df.loc[df['overall'] == 5]

In [7]:
review_text = df[['reviewText','sentiment']]

In [8]:
review_text

,reviewText,sentiment
0,I enjoy vintage books and movies so I enjoyed ...,pos
1,This book is a reissue of an old one; the auth...,pos
2,This was a fairly interesting read. It had ol...,pos
3,I'd never read any of the Amy Brewster mysteri...,pos
4,"If you like period pieces - clothing, lingo, y...",pos
...,...,...
982614,Yasss hunny! This is a great read. That Dre is...,pos
982615,I ENJOYED THIS BOOK FROM BEGINNING TO END NOW ...,pos
982616,Great book! Cherika was a fool. She let that m...,pos
982617,When I say this was an excellent book please b...,pos


In [9]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

2023-04-11 15:49:20.119075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.3 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
review_text_copy = review_text.copy()
review_text_copy['reviewText'] = review_text_copy['reviewText'].astype(str)

In [24]:
review_text_5000 = review_text_copy.iloc[:5000,:]

常见的预处理操作：
分词（tokenization）：将文本分成单独的单词或标记（token）。  
去除停用词（stop words removal）：去除一些常见但无意义的词语（如 "the"、"a"、"an" 等）。  
词干化（stemming）和词形还原（lemmatization）：将单词的不同形式（如复数形式、时态等）归一化为一个基本形式。  
去除标点符号（punctuation removal）：去除文本中的标点符号，这些符号通常不会提供太多的信息。  
去除 HTML 标签（HTML tag removal）：如果你的文本包含 HTML 标记，则需要去除它们，以便更好地处理文本。  
去除特殊字符（special character removal）：如果你的文本包含一些特殊字符（如 URL、邮件地址等），可以考虑去除它们。  

In [25]:
# 下载停用词列表和词干化器
nltk.download('stopwords')
nltk.download('punkt')

# 创建停用词列表
stop_words = stopwords.words('english')

# 创建词干化器
stemmer = SnowballStemmer('english')

# 定义预处理函数
def preprocess(text):
    
    # 将文本转换为小写
    text = text.lower()

    # 分词
    words = nltk.word_tokenize(text)

    # 去除停用词
    words = [word for word in words if word not in stop_words]

    # 词干化
    words = [stemmer.stem(word) for word in words]

    # 返回预处理后的文本
    return ' '.join(words)

# 对每行进行预处理
review_text_5000['processed_text'] = review_text_5000['reviewText'].apply(preprocess)

# 打印预处理后的结果
print(review_text_5000['processed_text'])


[nltk_data] Downloading package stopwords to /Users/Link/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Link/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0       enjoy vintag book movi enjoy read book . plot ...
1       book reissu old one ; author born 1910. 's era...
2       fair interest read . old- style terminology.i ...
3       'd never read ami brewster mysteri one .. real...
4       like period piec - cloth , lingo , enjoy myste...
                              ...                        
4995    good stuff : terrif story.th write skill seen ...
4996    short enough read chili cook . keep interest b...
4997    end , expect , done . n't expect much .99cent ...
4998    say , mix feel short . although initiali start...
4999    bought novel time ago 's sit kindl gather edus...
Name: processed_text, Length: 5000, dtype: object


/var/folders/qm/jh9yg7h129q058z58zlc62v80000gn/T/ipykernel_45897/1023774446.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_text_5000['processed_text'] = review_text_5000['reviewText'].apply(preprocess)


In [26]:
review_text_5000

,reviewText,sentiment,processed_text
0,I enjoy vintage books and movies so I enjoyed ...,pos,enjoy vintag book movi enjoy read book . plot ...
1,This book is a reissue of an old one; the auth...,pos,book reissu old one ; author born 1910. 's era...
2,This was a fairly interesting read. It had ol...,pos,fair interest read . old- style terminology.i ...
3,I'd never read any of the Amy Brewster mysteri...,pos,'d never read ami brewster mysteri one .. real...
4,"If you like period pieces - clothing, lingo, y...",pos,"like period piec - cloth , lingo , enjoy myste..."
...,...,...,...
4995,The Good Stuff:This was a terrific story.The w...,pos,good stuff : terrif story.th write skill seen ...
4996,Short enough to read while your chili is cooki...,pos,short enough read chili cook . keep interest b...
4997,"The ending, while not expected, has been done ...",neg,"end , expect , done . n't expect much .99cent ..."
4998,"I have to say, I have mixed feelings about thi...",neg,"say , mix feel short . although initiali start..."


In [27]:
nlp = spacy.load('en_core_web_sm')
# 创建语料库
corpus = st.CorpusFromPandas(review_text_5000, category_col='sentiment', text_col='processed_text', nlp=nlp).build()


In [40]:
corpus.get_df()

,level_0,index,reviewText,sentiment,processed_text
0,0,0,I enjoy vintage books and movies so I enjoyed ...,pos,enjoy vintag book movi enjoy read book . plot ...
1,1,1,This book is a reissue of an old one; the auth...,pos,book reissu old one ; author born 1910. 's era...
2,2,2,This was a fairly interesting read. It had ol...,pos,fair interest read . old- style terminology.i ...
3,3,3,I'd never read any of the Amy Brewster mysteri...,pos,'d never read ami brewster mysteri one .. real...
4,4,4,"If you like period pieces - clothing, lingo, y...",pos,"like period piec - cloth , lingo , enjoy myste..."
...,...,...,...,...,...
4995,4995,4995,The Good Stuff:This was a terrific story.The w...,pos,good stuff : terrif story.th write skill seen ...
4996,4996,4996,Short enough to read while your chili is cooki...,pos,short enough read chili cook . keep interest b...
4997,4997,4997,"The ending, while not expected, has been done ...",neg,"end , expect , done . n't expect much .99cent ..."
4998,4998,4998,"I have to say, I have mixed feelings about thi...",neg,"say , mix feel short . although initiali start..."


In [29]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['pos_score'] = corpus.get_scaled_f_scores('pos')
pos_most = list(term_freq_df.sort_values(by='pos_score',ascending=False).index[:100])

In [30]:
term_freq_df['neg_score'] = corpus.get_scaled_f_scores('neg')
neg_most = list(term_freq_df.sort_values(by='neg_score',ascending=False).index[:100])

In [31]:
term_freq_df

,pos freq,neg freq,pos_score,neg_score
term,,,,
enjoy,1233,250,0.939539,0.060461
vintag,4,0,0.558011,0.441989
book,4272,1538,0.158797,0.841203
movi,94,25,0.916714,0.083286
read,3698,1037,0.911641,0.088359
...,...,...,...,...
statement place,1,0,0.507501,0.492499
back fun,1,0,0.507501,0.492499
let author,1,0,0.507501,0.492499


In [32]:
print('Positive words:',pos_most)
print('Negative words:',neg_most)

Positive words: ['must read', "n't wait", 'bella', 'joy', 'high recommend', 'great read', 'morgan', 'joy review', 'love book', 'delight', 'wait read', 'zsadist', 'excel', 'lisa', 'love charact', "n't put", 'cat', 'amaz', 'emma', 'vampir', 'puma', 'jackson', 'jon', 'grow', 'enjoy read', 'realli enjoy', 'great book', 'fantast', 'thorough enjoy', 'awesom', 'truli', 'well worth', 'angel', 'thorough', 'ms', 'touch', 'bell', 'fairi tale', 'favorit', 'brenda', 'shane', 'insid', 'fairi', 'one favorit', 'love', 'ms .', 'captiv', 'pride', 'classic', 'blood', 'max', 'rafe', 'perfect', 'daughter', 'map', 'home', 'look forward', 'secret', 'great stori', 'read next', 'paranorm', 'love read', 'laugh', 'cole', 'put book', 'fascin', 'passion', 'happi', 'wolf', 'fun read', 'collect', 'visit', 'luke', 'great', 'bodi', 'known', 'pleasur', 'brought', 'easi', 'sexi', 'ago', 'lover', 'brett', 'men', 'alway', 'rachel', 'start read', 'pack', 'forward', 'wonder', 'wait', 'jule', 'heart', 'soul', '.', 'edg', 'ha

In [33]:
html = st.produce_scattertext_explorer(corpus, category='pos', category_name='pos', not_category_name='neg',
                                      width_in_pixels=1000,metadata=review_text_5000['sentiment'])
open("scattertext_01.html",'wb').write(html.encode('utf-8'))


3321654

In [41]:
from IPython.display import IFrame
html_path = '/Users/Link/Documents/5005数据可视化/Group project/archive/scattertext_01.html'
iframe = IFrame(src=html_path, width=1300, height=700)
display(iframe)